In [1]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = "ConicalClassificationGithub"
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd drive/My\ Drive
%cd $FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/ConicalClassificationGithub


In [2]:
from sklearn.datasets import load_files
import codecs as cs
from load_dataset import *

X_train, y_train, X_valid, y_valid, X_test, y_test = load_and_split()

In [3]:
from conical import CorpusClassification
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_FAIL
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample
import pickle

def load_model():
    space = {
        'remove_outliers': hp.choice('remove_outliers', [True, False]),
        'use_bns': hp.choice('use_bns', [True, False]),
        'final_norm': hp.choice('final_norm', ['l1', 'l2', 'max', None]),
        'norm': hp.choice('norm', ['l1', 'l2', None]),
        'sublinear_tf': hp.choice('sublinear_tf', [True, False]),
        'max_features': ho_scope.int(hp.quniform('max_features', 1, 10000, q=1)),

        'idf': hp.choice('idf', [
            {
                'use_idf': True,
                'smooth_idf': hp.choice('smooth_idf', [True, False]),
            },
            {
                'use_idf': False,
            }
        ])
    }
    trials = pickle.load(open("Conical_Demo.hyperopt","rb"))
    best_trial = {}
    for key in trials.best_trial['misc']['vals']:
        try:
            best_trial[key] = trials.best_trial['misc']['vals'][key][0]
        except:
            best_trial[key] = 0
    argDict = space_eval(space, best_trial)
    idfDict = argDict.pop("idf")
    for key in idfDict:
        argDict[key] = idfDict[key]
    return CorpusClassification(argDict)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
model = load_model()
model.fit(X_train)

In [5]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import *

y_pred = model.predict(X_valid)
print(f"Validation accuracy: {accuracy_score(y_valid, y_pred)}")

y_pred = model.predict(X_test)
print(f"Testing accuracy: {accuracy_score(y_test, y_pred)}")

Validation accuracy: 1.0
Testing accuracy: 1.0
